In [ ]:
import numpy as np
import pandas as pd
import re
from xgboost import XGBClassifier
from sklearn import metrics

In [ ]:
df_train=pd.read_csv("/kaggle/input/playground-series-s3e17/train.csv")
df_test=pd.read_csv("/kaggle/input/playground-series-s3e17/test.csv")

In [ ]:
df_train.isna().sum()

In [ ]:
df_Type = pd.get_dummies(df_train['Type'])
df_new = pd.concat([df_train, df_Type], axis=1)
print(df_new)

In [ ]:
df_train.drop(columns=['Product ID','Type'],axis=1,inplace=True)

In [ ]:
#Separate Target Variable and Predictor Variables
TargetVariable='Machine failure'
X=df_train.drop(TargetVariable,axis=1)
y=df_train[TargetVariable].values
 
#Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Removed slaches and commas
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

X_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]
X_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]

In [ ]:
clf=XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=500, objective='binary:logistic', booster='gbtree')
 
#Printing all the parameters of XGBoost
print(clf)

In [ ]:
#Creating the model on Training Data
XGB=clf.fit(X_train,y_train)
prediction=XGB.predict(X_test)
 
#Measuring accuracy on Testing Data
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))


In [ ]:
#Plotting the feature importance for Top 10 most important columns
%matplotlib inline
feature_importances = pd.Series(XGB.feature_importances_)
feature_importances.nlargest(10).plot(kind='barh')
 
#Printing some sample values of prediction
Predictors=[]
TestingDataResults=pd.DataFrame(data=X_test)
TestingDataResults['TargetColumn']=y_test
TestingDataResults['Prediction']=prediction
TestingDataResults.head()

In [ ]:
Predictors=[]
TestingDataResults=pd.DataFrame(data=X_test)
TestingDataResults['Machine failure']=prediction
sumbit=TestingDataResults.iloc[:,[0,-1]]
sumbit

In [ ]:
test=pd.read_csv("/kaggle/input/playground-series-s3e17/sample_submission.csv")
test.to_csv('submission.csv', index=None)